
<center><b>© Content is made available under the CC-BY-NC-ND 4.0 license. Christian Lopez, lopezbec@lafayette.edu<b><center>

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/lopezbec/intro_python_notebooks/blob/master/NN_with_Keras_GPUs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
  <td>
         </table>
          <br><br></br>

#Training NN with GPUs

This Notebook will give you an introduction on how to train a Neural Network using GPU in Google Colab.  

In this Notebook you will learn:

- The benefits of using GPUs to train NN
- Think to consider when training with GPUs
- The value of Deep NN




###**Before we start:**
Make sure you are using GPUs as your Hardware accelerator, go to `Runtime`>`Change runtime type`> `Run time type`.


In Google Colab is extremely easy to enable a GPU and train with it. However, if you have a CUDA enable GPU in your computer (or in a server) and you would like to use it instead, the process is a bit more complex (see a [tutorial here](https://medium.com/@ab9.bhatia/set-up-gpu-accelerated-tensorflow-keras-on-windows-10-with-anaconda-e71bfa9506d1)).


Most of the notebooks we are going to be using are inspired from existing notebooks that are available online and are made free for educational purposes. Nonetheless, these notebooks should not be share without prior permission of the instructor. When working in an assignment always remember the [Student Code of Conduct](https://conduct.lafayette.edu/student-handbook/student-code-of-conduct/).


# Setup

In [ ]:
!pip install --upgrade tensorflow
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

# TensorFlow ≥2.0 is required
import tensorflow as tf
assert tf.__version__ >= "2.0"
from tensorflow import keras

# Common imports
import numpy as np
import pandas as pd
import os

# to make this notebook's output stable across runs
np.random.seed(42)

import time

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

from IPython.display import clear_output

# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")
import logging

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # FATAL
logging.getLogger('tensorflow').setLevel(logging.FATAL)

!pip install Pillow
from PIL import Image
import imageio

# GPU memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
clear_output()

## 1- Google Colab GPU information



You might encounter more issues and difficulties when using GPU in Colab than CPU. You might need to `Factory reset runtime` and/or `Restart runtime` more frequently than before, or manually go over the code cell and run them `Shift+ENTER` (a small price to paid for the convenience of a free GPU).

Let's check how many GPUs do we have:



In [ ]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

What type of CPU and GPU do we have:

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

The default GPU for Google Colab is an [NVIDIA Tesla K80]( https://www.nvidia.com/en-gb/data-center/tesla-k80/)....but the [legend says]( https://www.reddit.com/r/MachineLearning/comments/duds5d/d_colab_has_p100_gpus/) that once in a blue moons the rare [NVIDIA P100]( https://www.nvidia.com/en-us/data-center/tesla-p100/) might show up and help you train your NN.

Let’s look at how much memory we have in our VM and GPU. This will ultimately constraint how many training samples we can load at once to the GPU to train our models.  

In [ ]:
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("VM RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.2f}GB | Used: {3:.2f} GB | Util {2:3.2f}% | Total {3:.2f} GB".format(gpu.memoryFree/1000, gpu.memoryUsed/1000, gpu.memoryUtil*100, gpu.memoryTotal/1000))
printm()

## 2-  Training on the MNIST Fashion dataset

We will use the same dataset as before to help us compare the speed of CPU vs GPU. Let's start by loading the fashion MNIST dataset.

In [ ]:
#load dataset
fashion_mnist = keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

#Split and Scale dataset
X_valid, X_train = X_train_full[:5000] / 255., X_train_full[5000:] / 255.
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
X_test = X_test / 255.

#Here are the corresponding class names:
class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
               "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

clear_output()

Let's take a look at a sample of the images in the dataset:

In [ ]:
n_rows = 4
n_cols = 10
plt.figure(figsize=(n_cols * 1.2, n_rows * 1.2))
for row in range(n_rows):
    for col in range(n_cols):
        index = n_cols * row + col
        plt.subplot(n_rows, n_cols, index + 1)
        plt.imshow(X_train[index], cmap="binary", interpolation="nearest")
        plt.axis('off')
        plt.title(class_names[y_train[index]], fontsize=12)
plt.subplots_adjust(wspace=0.2, hspace=0.5)
plt.show()

### 2.1- Building, Training, and Testing L=3 NN model

This is the same NN architecture, and training process we used in the last notebook. It will still take some time, but not as much as with a CPU!!

In [ ]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

# specify our Neural Network architecture
model_f = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])

#Compile the  model
model_f.compile(loss="sparse_categorical_crossentropy",
              optimizer="sgd",
              metrics=["accuracy"])


#Lets save our initial model weights so we can "reset" the traning latter
Wsave = model_f.get_weights().copy()

print(model_f.summary())
#lets plot the model info
keras.utils.plot_model(model_f, show_shapes=True)


In [ ]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

model_f.set_weights(Wsave)

#Train the model
tic = time.process_time()
history_nn_f = model_f.fit(X_train, y_train, epochs=30, batch_size=50, validation_data=(X_valid, y_valid), verbose=1)
toc = time.process_time()
#Model evaluation
test_nn_f=model_f.evaluate(X_test, y_test, verbose=0)

#Plot performnace
pd.DataFrame(history_nn_f.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1)
plt.show()

time_nn_f=toc-tic
print("Time to train the model: {0:.4f} secs ".format((time_nn_f)))
print("Training Set Accuracy:   {0:.4%} & Loss:{1:.4f}".format(history_nn_f.history['accuracy'][29],history_nn_f.history['loss'][29]))
print("Validation Set Accuracy: {0:.4%} & Loss:{1:.4f}".format(history_nn_f.history['val_accuracy'][29],history_nn_f.history['val_loss'][29]))
print("Test Set Accuracy:       {0:.4%} & Loss:{1:.4f}".format(test_nn_f[1],test_nn_f[0]))


If you compare these results with the results from our previous notebook, where we used CPU to train this same NN architecture, you will notice:
- The time to train the model was cut by more than half
- The Accuracy and Lost are close to the CPU results, but not the same.

If you restart the environment and run all the code cell above (`Runtime`>` Restart runtime…`>`Run before`)You will notice that you get some results a bit difference time over time, even if we are setting `np.random.seed(42)` and `tf.random.set_seed(42)`. This is because the GPU might be using some *“sophisticated stack of GPU libraries, and some of these may introduce their own source of randomness that you may or may not be able to account“* ( [see Randomness from Using the GPU)]( https://machinelearningmastery.com/reproducible-results-neural-networks-keras/)


### 2.2- Building, Training, and Testing Softmax Regression as a NN

Let see how would a simple Softmax Regression model will perform in this dataset

In [ ]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

# specify our Neural Network architecture
Log_model_f = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(10, activation="softmax")
])

#Compile the  model
Log_model_f.compile(loss="sparse_categorical_crossentropy",
              optimizer="sgd",
              metrics=["accuracy"])




#Lets save our initial model weights so we can "reset" the traning latter
Wsave = Log_model_f.get_weights().copy()

print(Log_model_f.summary())
#lets plot the model info
keras.utils.plot_model(Log_model_f, show_shapes=True)

In [ ]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

Log_model_f.set_weights(Wsave)

#Train the model
tic = time.process_time()
history_log_f = Log_model_f.fit(X_train, y_train, epochs=30, batch_size=50, validation_data=(X_valid, y_valid), verbose=0)
toc = time.process_time()
#Model evaluation
test_log_f=Log_model_f.evaluate(X_test, y_test, verbose=0)

#Plot performnace
import pandas as pd
pd.DataFrame(history_log_f.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1)
plt.show()

time_log_f=toc-tic
print("Time to train the model: {0:.4f} secs ".format((time_log_f)))
print("Training Set Accuracy:   {0:.4%} & Loss:{1:.4f}".format(history_log_f.history['accuracy'][29],history_log_f.history['loss'][29]))
print("Validation Set Accuracy: {0:.4%} & Loss:{1:.4f}".format(history_log_f.history['val_accuracy'][29],history_log_f.history['val_loss'][29]))
print("Test Set Accuracy:       {0:.4%} & Loss:{1:.4f}".format(test_log_f[1],test_log_f[0]))



If you compare these results with the previous L=3 NN results, a model that had 250k more parameters, you will notice:

- The time to train is less, but not by much
- The model performed a bit worse, but not much worse

This is because the NN take advantage of the GPU ability to perform matrix and vector multiplication. Lastly, the Softmax Regression is an enough complex model to capture most of the variability in the dataset (i.e., patterns of the dataset). You could even use a TPU to train a more complex model faster, but you need to update the code bit (see [tutorial here](https://heartbeat.fritz.ai/step-by-step-use-of-google-colab-free-tpu-75f8629492b3))


## 3-  Training on the MNIST Digits dataset

In [ ]:

#load dataset
fashion_mnist = keras.datasets.mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

#Split and Scale dataset
X_valid, X_train = X_train_full[:5000] / 255., X_train_full[5000:] / 255.
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
X_test = X_test / 255.

#Here are the corresponding class names:
class_names = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]

clear_output()

In [ ]:
y_train.shape

In [ ]:
n_rows = 4
n_cols = 10
plt.figure(figsize=(n_cols * 1.2, n_rows * 1.2))
for row in range(n_rows):
    for col in range(n_cols):
        index = n_cols * row + col
        plt.subplot(n_rows, n_cols, index + 1)
        plt.imshow(X_train[index], cmap="binary", interpolation="nearest")
        plt.axis('off')
        plt.title(class_names[y_train[index]], fontsize=12)
plt.subplots_adjust(wspace=0.2, hspace=0.5)
plt.show()

### 3.1- Building, Training, and Testing L=3 NN model

In [ ]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

# specify our Neural Network architecture
model_nn = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])

#Compile the  model
model_nn.compile(loss="sparse_categorical_crossentropy",
              optimizer="sgd",
              metrics=["accuracy"])


#Lets save our initial model weights so we can "reset" the traning latter
Wsave = model_nn.get_weights().copy()

In [ ]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

model_nn.set_weights(Wsave)

#Train the model
tic = time.process_time()
history_nn = model_nn.fit(X_train, y_train, epochs=30, batch_size=50, validation_data=(X_valid, y_valid), verbose=0)
toc = time.process_time()
#Model evaluation
test_nn=model_nn.evaluate(X_test, y_test, verbose=0)

#Plot performnace
pd.DataFrame(history_nn.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1)
plt.show()

time_nn=toc-tic
print("Time to train the model: {0:.4f} secs ".format((time_nn)))
print("Training Set Accuracy:   {0:.4%} & Loss:{1:.4f}".format(history_nn.history['accuracy'][29],history_nn.history['loss'][29]))
print("Validation Set Accuracy: {0:.4%} & Loss:{1:.4f}".format(history_nn.history['val_accuracy'][29],history_nn.history['val_loss'][29]))
print("Test Set Accuracy:       {0:.4%} & Loss:{1:.4f}".format(test_nn[1],test_nn[0]))


### 3.2- Building, Training, and Testing Softmax Regression as a NN

In [ ]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

# specify our Neural Network architecture
Log_model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(10, activation="softmax")
])

#Compile the  model
Log_model.compile(loss="sparse_categorical_crossentropy",
              optimizer="sgd",
              metrics=["accuracy"])




#Lets save our initial model weights so we can "reset" the traning latter
Wsave = Log_model.get_weights().copy()


In [ ]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

Log_model.set_weights(Wsave)

#Train the model
tic = time.process_time()
history_log = Log_model.fit(X_train, y_train, epochs=30, batch_size=50, validation_data=(X_valid, y_valid), verbose=0)
toc = time.process_time()
#Model evaluation
test_log=Log_model.evaluate(X_test, y_test, verbose=0)

#Plot performnace
import pandas as pd
pd.DataFrame(history_log.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1)
plt.show()

time_log=toc - tic
print("Time to train the model: {0:.4f} secs ".format((time_log)))
print("Training Set Accuracy:   {0:.4%} & Loss:{1:.4f}".format(history_log.history['accuracy'][29],history_log.history['loss'][29]))
print("Validation Set Accuracy: {0:.4%} & Loss:{1:.4f}".format(history_log.history['val_accuracy'][29],history_log.history['val_loss'][29]))
print("Test Set Accuracy:       {0:.4%} & Loss:{1:.4f}".format(test_log[1],test_log[0]))


# Now I want you to look at all the results and think:
-	Why the time to train the models did not change as much between the dataset
-	Why the L=3 NN and Softmax Regression performed better in the Digits dataset


In [ ]:
print("                   MNIST FASHION DATASET")
print("   L3 NN: n[1]=300 n[2]=100")
print("Time to train the model: {0:.4f} secs ".format((time_nn_f)))
print("Training Set Accuracy:   {0:.4%} & Loss:{1:.4f}".format(history_nn_f.history['accuracy'][29],history_nn_f.history['loss'][29]))
print("Validation Set Accuracy: {0:.4%} & Loss:{1:.4f}".format(history_nn_f.history['val_accuracy'][29],history_nn_f.history['val_loss'][29]))
print("Test Set Accuracy:       {0:.4%} & Loss:{1:.4f}".format(test_nn_f[1],test_nn_f[0]))
print("----------------------------------------------------------------------------------")
print("    SoftMax")
print("Time to train the model: {0:.4f} secs ".format((time_log_f)))
print("Training Set Accuracy:   {0:.4%} & Loss:{1:.4f}".format(history_log_f.history['accuracy'][29],history_log_f.history['loss'][29]))
print("Validation Set Accuracy: {0:.4%} & Loss:{1:.4f}".format(history_log_f.history['val_accuracy'][29],history_log_f.history['val_loss'][29]))
print("Test Set Accuracy:       {0:.4%} & Loss:{1:.4f}".format(test_log_f[1],test_log_f[0]))
print("\n")

print("                  MNIST DIGITS DATASET")
print("   L3 NN: n[1]=300 n[2]=100")
print("Time to train the model: {0:.4f} secs ".format((time_nn)))
print("Training Set Accuracy:   {0:.4%} & Loss:{1:.4f}".format(history_nn.history['accuracy'][29],history_nn.history['loss'][29]))
print("Validation Set Accuracy: {0:.4%} & Loss:{1:.4f}".format(history_nn.history['val_accuracy'][29],history_nn.history['val_loss'][29]))
print("Test Set Accuracy:       {0:.4%} & Loss:{1:.4f}".format(test_nn[1],test_nn[0]))
print("----------------------------------------------------------------------------------")
print("    SoftMax")
print("Time to train the model: {0:.4f} secs ".format((time_log)))
print("Training Set Accuracy:   {0:.4%} & Loss:{1:.4f}".format(history_log.history['accuracy'][29],history_log.history['loss'][29]))
print("Validation Set Accuracy: {0:.4%} & Loss:{1:.4f}".format(history_log.history['val_accuracy'][29],history_log.history['val_loss'][29]))
print("Test Set Accuracy:       {0:.4%} & Loss:{1:.4f}".format(test_log[1],test_log[0]))

## 4-  Other Keras and TensorFlow features




There are a lot of interesting features from Keras and TensorFLow that make it very easy to train large NN and keep track of everything. For example:

###  4.1 -Builing larger NN using loops

In [ ]:
# Import TensorFlow and check version for confirmation
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

# Import necessary modules from tensorflow.keras
from tensorflow.keras import regularizers, backend, models, layers

# Instead of importing from tensorflow.keras.wrappers.scikit_learn, use scikeras:
from scikeras.wrappers import KerasClassifier

# Clear previous sessions and set seeds for reproducibility
backend.clear_session()
import numpy as np
np.random.seed(42)
tf.random.set_seed(42)

# Define the model building function
def build_model(n_hidden=20, n_neurons=20, reg=0, input_shape=[28, 28]):
    model = models.Sequential()
    # Flatten the input (e.g., 28x28 image) to a 1D vector
    model.add(layers.Flatten(input_shape=input_shape))
    # Add hidden Dense layers with ReLU activation and L2 regularization
    for _ in range(n_hidden):
        model.add(layers.Dense(n_neurons, activation="relu",
                               kernel_regularizer=regularizers.l2(reg)))
    # Output layer for 10 classes with softmax activation
    model.add(layers.Dense(10, activation="softmax"))

    # Compile the model with SGD optimizer and sparse categorical crossentropy loss
    optimizer = "sgd"
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# Wrap the model with scikeras' KerasClassifier for scikit-learn compatibility
keras_reg = KerasClassifier(model=build_model)


###  4.2 -Adding "callbacks"

Adding a [Keras callbacks](https://keras.io/api/callbacks/) to your models would allow you to do a lot of different things, like keep a log of your training process just in case your systems crashes, save the best model so you can share it with others, and use TensorBoard to visualize your model process as it trains





In [ ]:
import os
import time
import tensorflow as tf
from tensorflow import keras

# Using CIFAR-10 dataset as an example. It has images of shape (32, 32, 3).
(X_train, y_train), (X_valid, y_valid) = keras.datasets.cifar10.load_data()

# Normalize pixel values to the range [0, 1]
X_train = X_train.astype("float32") / 255.
X_valid = X_valid.astype("float32") / 255.


# Create a Sequential model with an input layer that matches the CIFAR-10 image shape.
model = keras.models.Sequential([
    keras.layers.Input(shape=(32, 32, 3)),  # Matching the input shape of CIFAR-10 images
    keras.layers.Conv2D(32, (3, 3), activation="relu"),
    keras.layers.MaxPooling2D(2, 2),
    keras.layers.Conv2D(64, (3, 3), activation="relu"),
    keras.layers.MaxPooling2D(2, 2),
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation="relu"),
    keras.layers.Dense(10, activation="softmax")  # 10 classes for CIFAR-10
])

# Compile the model
model.compile(optimizer="adam",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])


# Create a directory for saving log files (if it doesn't exist)
root_logdir = os.path.join(os.curdir, "my_logs")
if not os.path.exists(root_logdir):
    os.makedirs(root_logdir)

def get_run_logdir():
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)

run_logdir = get_run_logdir()

# Callback to save the best model during training
checkpoint_cb = keras.callbacks.ModelCheckpoint("my_keras_model.h5", save_best_only=True)

# TensorBoard callback to log training events (adjust log directory)
tensorboard_cb = keras.callbacks.TensorBoard(log_dir=run_logdir)


history = model.fit(
    X_train, y_train,
    epochs=10,
    batch_size=50,
    validation_data=(X_valid, y_valid),
    verbose=1,
    callbacks=[checkpoint_cb, tensorboard_cb]
)




###  4.3 -[TensorBoard](https://keras.io/api/callbacks/tensorboard/)

In [ ]:

%load_ext tensorboard
%tensorboard --logdir=./my_logs --port=6006

## 5-  Training on a color image dataset

In [ ]:

#load dataset
cifar10 = keras.datasets.cifar10
(X_train_full, y_train_full), (X_test, y_test) = cifar10.load_data()
y_train_full=y_train_full.ravel()
y_test=y_test.ravel()
#Split and Scale dataset
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
X_test = X_test

#Here are the corresponding class names:
class_names = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]

clear_output()


In [ ]:
n_rows = 4
n_cols = 4
plt.figure(figsize=(n_cols * 3, n_rows * 3))
for row in range(n_rows):
    for col in range(n_cols):
        index = n_cols * row + col
        plt.subplot(n_rows, n_cols, index + 1)
        plt.imshow(X_train[index], interpolation="nearest")
        plt.axis('off')
        plt.title(class_names[y_train[index]], fontsize=12)
plt.subplots_adjust(wspace=0.2, hspace=0.5)
plt.show()

#### **Exercise**:

Now use the [CIFAR10](https://keras.io/datasets/) to train a NN of your choice. Make sure to:

-	Make sure your model does not take more than 5mins to train (using GPU) and print the training time (use `tic = time.process_time()` `toc = time.process_time()`    )
- Make sure to print the Accuracy and Loss for each of the datasets
-	Make sure to plot the Loss and performance metric of your model
-Explain in a text cell how is your model performing (over/underfitting)


In [ ]:
############################# START CODE HERE #############################







############################# END CODE HERE #############################

### Explain in a text cell how is your model performing (over/underfitting) and what you should do:


### You can now test the model in your own image

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:

#(PUT YOUR IMAGE NAME)
fname = "_________.jpg"   # change this to the name of your image file

# We preprocess the image to fit your algorithm (*64*64 pixels).

img = Image.open(fname)
img.show()
new_image = img.resize((32, 32))
data = np.asarray(new_image, dtype="int32" )

plt.figure()
plt.imshow(data)


img = Image.open(fname)
new_image = img.resize((32, 32))
data = np.asarray(new_image, dtype="float64" )
data=np.array([data/255.])
prediction=model.predict(data)
print("predicted class:  " +class_names[np.argmax(prediction)] )
